## **imports**

In [167]:
from dotenv import load_dotenv
import os
import yaml

from langchain_google_genai import GoogleGenerativeAIEmbeddings, GoogleGenerativeAI

## **setting env variables**

In [227]:
env_path = r"D:\common_credentials\.env"
load_dotenv(dotenv_path=env_path)

GROQ_API_KEY = os.getenv('GROQ_API_KEY')
GEMINI_API_KEY = os.getenv("GEMINI_API_KEY")
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")

os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY
os.environ["GEMINI_API_KEY"] = GEMINI_API_KEY
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGSMITH_PROJECT"]=os.getenv("LANGSMITH_PROJECT")
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGSMITH_PROJECT")
os.environ["LANGSMITH_ENDPOINT"]=os.getenv("LANGSMITH_ENDPOINT")

In [228]:
os.getenv("LANGSMITH_PROJECT"), os.getenv("LANGCHAIN_PROJECT")

('pr-advanced-chess-51', 'pr-advanced-chess-51')

## **load config yaml file**

In [178]:
with open("../config.yml", "r") as yml_file:
    yml_config= yaml.safe_load(yml_file)
    yml_file.close()
yml_config

{'googleModels': {'Embedding_models': ['models/embedding-001',
   'models/text-embedding-004'],
  'Inferencing_Models': {'text_models': ['gemini-pro'],
   'multimodels': ['gemini-2.0-flash-exp',
    'gemini-1.5-flash',
    'gemini-1.5-flash-8b']}},
 'GroqModels': {'Inferencing_Models': {'multimodels': ['llama-3.2-90b-vision-preview',
    'llama-3.2-11b-vision-preview'],
   'reasoning_Models': ['deepseek-r1-distill-llama-70b'],
   'text_models': ['distil-whisper-large-v3-en',
    'gemma2-9b-it',
    'llama-3.3-70b-versatile',
    'llama-3.1-8b-instant',
    'whisper-large-v3-turbo',
    'deepseek-r1-distill-llama-70b']}}}

## groq apis

In [68]:
from groq import Groq

client = Groq()
completion = client.chat.completions.create(
    model="llama-3.2-11b-vision-preview",
    messages=[
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "What's in this image?"
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": "https://upload.wikimedia.org/wikipedia/commons/f/f2/LPU-v1-die.jpg"
                    }
                }
            ]
        }
    ],
    temperature=1,
    max_completion_tokens=1024,
    top_p=1,
    stream=False,
    stop=None,
)

print(completion.choices[0].message)


ChatCompletionMessage(content="The image appears to be a visual representation of the floorplan or layout of a large building, possibly an office or school. The image is divided into sections, with each section containing multiple rows of what appear to be rooms or classrooms. The rooms are arranged in a grid-like pattern, with some sections having more rows than others. Some sections have windows, which suggests that these rooms are on the upper levels of the building.\n\nThe image also features various architectural elements, such as stairwells, corridors, and possibly even a rooftop or outdoor area. The overall design of the building seems to be functional and efficient, with a focus on maximizing space and providing ample natural light.\n\nThe image may also include some additional features, such as doors, windows, and possibly even some decorative elements like railings or trim. The overall aesthetic of the building appears to be modern and sleek, with clean lines and minimal orna

In [69]:
import pprint

In [71]:
pprint.pprint(completion.choices[0].message.content)

('The image appears to be a visual representation of the floorplan or layout '
 'of a large building, possibly an office or school. The image is divided into '
 'sections, with each section containing multiple rows of what appear to be '
 'rooms or classrooms. The rooms are arranged in a grid-like pattern, with '
 'some sections having more rows than others. Some sections have windows, '
 'which suggests that these rooms are on the upper levels of the building.\n'
 '\n'
 'The image also features various architectural elements, such as stairwells, '
 'corridors, and possibly even a rooftop or outdoor area. The overall design '
 'of the building seems to be functional and efficient, with a focus on '
 'maximizing space and providing ample natural light.\n'
 '\n'
 'The image may also include some additional features, such as doors, windows, '
 'and possibly even some decorative elements like railings or trim. The '
 'overall aesthetic of the building appears to be modern and sleek, with c

## langchain

In [179]:
print("Groque moodels : ", yml_config["GroqModels"]["Inferencing_Models"]["text_models"])

Groque moodels :  ['distil-whisper-large-v3-en', 'gemma2-9b-it', 'llama-3.3-70b-versatile', 'llama-3.1-8b-instant', 'whisper-large-v3-turbo', 'deepseek-r1-distill-llama-70b']


In [180]:
from langchain_groq import ChatGroq
from pprint import  pprint

llm= ChatGroq(model="llama-3.3-70b-versatile")
response= llm.invoke("write 7 line poem on Gujarat")
pprint(response.content)

("In Gujarat's vibrant land of delight,\n"
 'The sun shines bright with a warm, golden light.\n'
 'The Gir forests whisper secrets of old,\n'
 'As lions roam free, their stories untold.\n'
 'The temples of Somnath stand tall and grand,\n'
 "A testament to the state's rich, cultural hand.\n"
 'In Gujarat, tradition and progress entwine.')


In [181]:
from langchain_core.prompts import ChatPromptTemplate

prompt= ChatPromptTemplate.from_messages(
    [("system", "You are an exper poet, who writes deep meaning poem"),
    ("user","{query}")]
)
print(prompt.input_variables)

chain = prompt | llm
response= chain.invoke({"query": "Gujarat"})
print(response.content)


['query']
"In the land of Gujarat's golden light,
Where the sun dips into the Arabian night,
A tapestry woven, rich and bold,
A heritage of tales, forever to be told.

The lions of Gir, with manes of might,
Roam free, a symbol of courage in sight,
The temples of Somnath, a testament to the past,
Echoes of a history that forever will last.

The Sabarmati's gentle flow, a river's song,
That whispers secrets, of a freedom long,
The ashram's humble walls, where Gandhi's spirit resides,
A beacon of hope, where love and peace abide.

In the vibrant streets of Ahmedabad's old town,
The scent of street food, a culinary crown,
The sound of the garba, a rhythmic beat,
A celebration of life, where hearts skip a treat.

The white salt desert, a surreal landscape wide,
The Rann of Kutch, where the moon glows with pride,
The folk tales of Saurashtra, a treasure to share,
A cultural richness, beyond compare.

Gujarat, a land of contrasts, a land of might,
Where tradition meets innovation, in the morn

In [126]:
template = ChatPromptTemplate([
    ("system", "You are a helpful AI bot. Your name is {name}."),
    ("human", "Hello, how are you doing?"),
    ("ai", "I'm doing well, thanks!"),
    ("human", "{user_input}"),
])

prompt_value = template.invoke(
    {
        "name": "Bob",
        "user_input": "What is your name?"
    }
)

print( "Prompt Value" ,prompt_value)

chain= template | llm
print("\nChain :", chain)

#invoke the chain now
response= chain.invoke({
        "name": "Bob",
        "user_input": "What is your name?"
    })

print("\nresponse", response)

print("\n response content: ", response.content)

Prompt Value messages=[SystemMessage(content='You are a helpful AI bot. Your name is Bob.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello, how are you doing?', additional_kwargs={}, response_metadata={}), AIMessage(content="I'm doing well, thanks!", additional_kwargs={}, response_metadata={}), HumanMessage(content='What is your name?', additional_kwargs={}, response_metadata={})]

Chain : first=ChatPromptTemplate(input_variables=['name', 'user_input'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=['name'], input_types={}, partial_variables={}, template='You are a helpful AI bot. Your name is {name}.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='Hello, how are you doing?'), additional_kwargs={}), AIMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}

In [136]:
## streming hepls to monitor the tokens and all
# treaming allows you to display the LLM's output in real-time, as it's being generated.
#  This makes your application feel more interactive and responsive, 
# as users don't have to wait for the entire response to be generated before seeing any results
response_tokens= []
for i in chain.stream({
        "name": "Bob",
        "user_input": "What is your name?"
    }):
    print(i.content)
    response_tokens.append(i.content)

print("".join(response_tokens))


My
 name
 is
 Bob
,
 nice
 to
 meet
 you
!
 I
'm
 here
 to
 help
 with
 any
 questions
 or
 tasks
 you
 might
 have
,
 so
 feel
 free
 to
 ask
 me
 anything
.

My name is Bob, nice to meet you! I'm here to help with any questions or tasks you might have, so feel free to ask me anything.


In [138]:
print("".join(response_tokens))

My name is Bob, nice to meet you! I'm here to help with any questions or tasks you might have, so feel free to ask me anything.


In [102]:
from langchain_core.prompts import PromptTemplate

In [108]:
prompt= PromptTemplate.from_template("share the poem of {query} in urdu words while font in hindi")
print(prompt.input_variables)

chain = prompt | llm
response= chain.invoke({"query": "Gulzar"})
print(response.content)

['query']
मैं आपको गुलज़ार साहब की एक प्रसिद्ध कविता सुनाना चाहता हूँ, जो उर्दू शब्दों में है लेकिन हिंदी फ़ॉन्ट में लिखी हुई है:


मुहब्बत करने वाले कम नहीं होते
नाम अक्सर लोग बदल लेते हैं
वो ख्वाब नहीं है जो ख्वाबों में मिले
वो सच है जो ख्वाबों में नहीं मिलता


मैंने इस कविता को हिंदी फ़ॉन्ट में लिखा है, लेकिन उर्दू शब्दों का उपयोग किया है। आशा है कि आपको यह पसंद आएगी।


यह कविता गुलज़ार साहब की एक प्रसिद्ध रचना है, जो मुहब्बत और सच्चाई के बारे में बात करती है।


In [146]:
#prompt with multiple variables
prompt= PromptTemplate.from_template("share the poem of {query} in urdu words in 4 lines while font in {language}")
print(prompt.input_variables)
chain = prompt | llm
response= chain.invoke({"query": "Gulzar", "language": "Marathi"})
print(response.content)


['language', 'query']
आपल्याला माहीत आहे की, गुलज़ार साहब हे एक प्रसिद्ध कवी आहेत. त्यांची काही पंक्ती अश्या आहेत:
गुलज़ार साहेबांची काव्य पंक्ती:
मैंने देखा है कभी तुम्हें 
मैंने देखा है कभी तुम्हें 
तुम्हारी आँखों में 
मैंने देखा है खुद को


# **Chains**

In [143]:
prompt= PromptTemplate.from_template("You are an expert who explain the {topic} in point by point lines, don't provide preamble")

#chain
chain=prompt|llm
response=chain.invoke({"topic":"Can you tell me something about Genertaive ai vs agentic ai"})
print(response.content)


* Generative AI focuses on creating new content, such as images, music, or text, based on patterns learned from existing data.
* Agentic AI, on the other hand, emphasizes the development of autonomous agents that can make decisions, take actions, and interact with their environment.
* Generative AI is primarily used for creative tasks, like generating art, music, or writing, whereas Agentic AI is used for tasks that require decision-making, problem-solving, and adaptability.
* Generative models, such as GANs and VAEs, are commonly used in Generative AI, whereas Agentic AI relies on architectures like reinforcement learning, deep learning, and cognitive architectures.
* Generative AI is often evaluated based on the quality, coherence, and diversity of the generated content, whereas Agentic AI is evaluated based on its ability to achieve goals, make optimal decisions, and adapt to changing situations.
* Generative AI can be used for applications like content creation, data augmentation, 

# **Output Parser**

## Stroutput Parser
The StrOutputParser is a fundamental component in the Langchain framework, designed to streamline the output from language models (LLMs) and ChatModels into a usable string format. This parser is particularly useful when dealing with outputs that may vary in structure, such as strings or messages. It ensures that the output is consistent and easy to handle in subsequent processing steps.

In [155]:
from langchain_core.output_parsers import StrOutputParser
prompt= PromptTemplate.from_template("You are an expert who explain the {topic} e")

#chain
chain=prompt| llm| StrOutputParser()
response=chain.invoke({"topic":"Can you tell me something about Genertaive ai vs agentic ai"})
response


'Generative AI and Agentic AI are two distinct types of artificial intelligence that have gained significant attention in recent years. While both types of AI have the potential to revolutionize various industries, they differ fundamentally in their goals, approaches, and applications.\n\n**Generative AI:**\n\nGenerative AI refers to a type of artificial intelligence that focuses on generating new, original content, such as text, images, music, or videos. The primary goal of generative AI is to create novel, coherent, and often realistic outputs that are similar to those produced by humans. Generative AI models, such as Generative Adversarial Networks (GANs) and Variational Autoencoders (VAEs), use complex algorithms to learn patterns and structures from large datasets and then generate new samples that resemble the training data.\n\nExamples of generative AI include:\n\n1. Text-to-text models that generate articles, stories, or chatbot responses.\n2. Image-to-image models that generat

In [205]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query with key.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain=prompt|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

{'name': 'Langsmith', 'description': 'Langsmith is an AI designed to simulate conversation, answer questions, and provide information on a wide range of topics.', 'key_features': ['Natural Language Processing', 'Conversational Interface', 'Knowledge Base'], 'capabilities': ['Answering user queries', 'Providing definitions and explanations', 'Engaging in conversation']}


In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
output_parser=JsonOutputParser()
prompt = PromptTemplate(
    template="Answer the user query with 'user ans' key.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": output_parser.get_format_instructions()},
)

chain=prompt|llm|output_parser

response=chain.invoke({"query":"Can you tell me about Langsmith?"})
print(response)

In [191]:
JsonOutputParser().get_format_instructions()

'Return a JSON object.'

## Json Output Parser

In [165]:
from langchain.output_parsers.json import SimpleJsonOutputParser

json_prompt = PromptTemplate.from_template(
    "Return a JSON object with an `Shubham` key that answers the following question: {question}"
)
json_parser = SimpleJsonOutputParser()
json_chain = json_prompt | llm | json_parser
json_chain

PromptTemplate(input_variables=['question'], input_types={}, partial_variables={}, template='Return a JSON object with an `Shubham` key that answers the following question: {question}')
| ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x000001DAC37A3D10>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x000001DAC37A1E90>, model_name='llama-3.3-70b-versatile', model_kwargs={}, groq_api_key=SecretStr('**********'))
| JsonOutputParser()

In [166]:
list(json_chain.stream({"question": "Who invented the microscope?"}))

[{},
 {'Shubham': ''},
 {'Shubham': 'The'},
 {'Shubham': 'The invention'},
 {'Shubham': 'The invention of'},
 {'Shubham': 'The invention of the'},
 {'Shubham': 'The invention of the microscope'},
 {'Shubham': 'The invention of the microscope is'},
 {'Shubham': 'The invention of the microscope is often'},
 {'Shubham': 'The invention of the microscope is often credited'},
 {'Shubham': 'The invention of the microscope is often credited to'},
 {'Shubham': 'The invention of the microscope is often credited to Zach'},
 {'Shubham': 'The invention of the microscope is often credited to Zacharias'},
 {'Shubham': 'The invention of the microscope is often credited to Zacharias J'},
 {'Shubham': 'The invention of the microscope is often credited to Zacharias Jans'},
 {'Shubham': 'The invention of the microscope is often credited to Zacharias Janssen'},
 {'Shubham': 'The invention of the microscope is often credited to Zacharias Janssen,'},
 {'Shubham': 'The invention of the microscope is often cre

In [184]:
SimpleJsonOutputParser().get_format_instructions()

'Return a JSON object.'

In [194]:
## Data Ingestion--From the website we need to scrape the data
from langchain_community.document_loaders import WebBaseLoader

In [195]:
loader=WebBaseLoader("https://python.langchain.com/docs/introduction/")
loader

In [198]:
documents=loader.load()
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='\n\n\n\n\nIntroduction | 🦜️🔗 LangChain\n\n\n\n\n\n\nSkip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a Retrieval Augmented Generation (RAG) App: Part 1Build a semantic search engineBuild a Question/Answering system over SQL dataSummarize TextHow-to guidesHow-to guidesHow to use tools in a chainHow to use a vectorstore as a retrieverHow to add memor

In [199]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
documents=text_splitter.split_documents(documents)
documents

[Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Introduction | 🦜️🔗 LangChain'),
 Document(metadata={'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain', 'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en'}, page_content='Skip to main contentIntegrationsAPI ReferenceMoreContributingPeopleError referenceLangSmithLangGraphLangChain HubLangChain JS/TSv0.3v0.3v0.2v0.1💬SearchIntroductionTutorialsBuild a Question Answering application over a Graph DatabaseTutorialsBuild a simple LLM application with chat models and prompt templatesBuild a ChatbotBuild a Retrieval Augmented Generation (RAG) App: Part 2Build an Extraction ChainBuild an AgentTaggingBuild a 

In [204]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
embeddings= GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
embeddings

GoogleGenerativeAIEmbeddings(client=<google.ai.generativelanguage_v1beta.services.generative_service.client.GenerativeServiceClient object at 0x000001DAD47DA110>, model='models/text-embedding-004', task_type=None, google_api_key=SecretStr('**********'), credentials=None, client_options=None, transport=None, request_options=None)

In [208]:
from langchain_community.vectorstores import Chroma
vectorstoredb=Chroma.from_documents(documents,embeddings)
vectorstoredb

In [210]:
query="Langchain is a framework"
result=vectorstoredb.similarity_search(query)
result

[Document(metadata={'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en', 'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'),
 Document(metadata={'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'language': 'en', 'source': 'https://python.langchain.com/docs/introduction/', 'title': 'Introduction | 🦜️🔗 LangChain'}, page_content='LangChain is a framework for developing applications powered by large language models (LLMs).\nLangChain simplifies every stage of the LLM application lifecycle:'),
 Document(metadata={'description': 'LangChain is a framework for developing applications powered by large language models (LLMs).', 'lan

# **LangSmith**

In [249]:
from langsmith import Client, traceable
langsmith_client = Client(api_key=os.getenv('LANGCHAIN_API_KEY'))
langsmith_client

Client (API URL: https://api.smith.langchain.com)

'pr-advanced-chess-51'

In [255]:
url = next(langsmith_client.list_runs(project_name=os.getenv("LANGSMITH_PROJECT"))).url
url

LangSmithNotFoundError: Project pr-advanced-chess-51 not found

In [250]:
from groq import Groq
@traceable(langsmith_client)
def interactwithGroq(question):
    groq_client = Groq()
    chat_completion = client.chat.completions.create(
    messages=[
        {   "role": "system",
            "content": "you are a helpful assistant."
        },
        {
            "role": "user",
            "content": f"{question}",
        }
    ],
    model="llama-3.3-70b-versatile",
    temperature=0.5)
    return chat_completion.choices[0].message.content

In [251]:
response= interactwithGroq("Tell me about jalgaon City.")
response

'Jalgaon is a city located in the state of Maharashtra, India. It is the administrative headquarters of the Jalgaon district and is situated on the northern bank of the Tapi River. Here are some key facts about Jalgaon city:\n\n1. **Location**: Jalgaon is located at an altitude of 209 meters (686 feet) above sea level, approximately 420 kilometers (261 miles) northeast of Mumbai, the state capital.\n2. **Population**: As of the 2011 census, the city has a population of around 460,000 people, making it one of the larger cities in Maharashtra.\n3. **Economy**: Jalgaon is a significant commercial center, with major industries including textiles, food processing, and manufacturing. The city is also known for its banana cultivation and is often referred to as the "Banana City of India."\n4. **Tourist attractions**: Jalgaon has several tourist attractions, including:\n\t* The Patna Devi temple, a historic temple dedicated to the goddess Durga.\n\t* The Omkareshwar temple, a Shiva temple loca